In [ ]:
import wave
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt 
import tflite_runtime.interpreter as tflite

In [ ]:
freq = 16000
dur = 2.0
recording = sd.rec(int(dur * freq), samplerate=freq, channels=1)
print("Recording started")
sd.wait()
print("Recording stopped")

In [ ]:
recording = np.squeeze(recording, axis = 1)

audio_normalised = recording
zero_padding = np.zeros(48000-np.shape(audio_normalised)[0])
equal_length = np.concatenate([audio_normalised, zero_padding], 0)

frame_length=255
frame_step=128

w = np.hanning(frame_length+1)
X = np.array([np.fft.rfft(w*equal_length[i:i+frame_length+1]) 
              for i in range(0, len(equal_length)-frame_length, frame_step)])

X = np.abs(X)
spectrogram = np.reshape(X, [1,374,129,1])
spectrogram = spectrogram.astype(np.float32)

In [ ]:
interpreter = tflite.Interpreter(model_path="path/to/tflite/model")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], spectrogram)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

fnl_out = output_data.flatten()
# fnl_out

In [ ]:

data = fnl_out*100

def index(i):
    switcher={
        0:'Backup',
        1:'Connection',
        2:'Device Diagnostic', 
        3:'Device Info',
        4:'Hello PDM',
        5:'No',
        6:'Reboot PDM',
        7:'Reboot to recovery mode',
        8:'Remove USB',
        9:'Run all',
        10:'Start local setup',
        11:'Stop local setup',
        12:'System setup',
        13:'Yes'
        
    }
    return switcher.get(i,"Invalid")

for i in range(len(data)):
    if(data[i]==max(data)):
        print("Prediction:",index(i))